# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

/Users/vietthangc1/projects/udemy/llm_engineering/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/gemma-3n-E4B-it',
 '/OmniGen2/OmniGen2',
 '/maya-research/Veena',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/multimodalart/self-forcing',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/facebook/seamless-interaction',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/OpenGVLab/MMBench-GUI',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tra

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
black-forest-labs/FLUX.1-Kont

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n5 days ago\n•\n122k\n•\n1.16k\ntencent/Hunyuan-A13B-Instruct\nUpdated\nabout 12 hours ago\n•\n6.96k\n•\n673\ngoogle/gemma-3n-E4B-it\nUpdated\n5 days ago\n•\n120k\n•\n373\nOmniGen2/OmniGen2\nUpdated\n9 days ago\n•\n32.1k\n•\n302\nmaya-research/Veena\nUpdated\n7 days ago\n•\n6.49k\n•\n151\nBrowse 1M+ models\nSpaces\nRunning\n9.05k\n9.05k\nDeepSi

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face
**The AI community building the future.** Hugging Face is the leading platform where the machine learning community comes together to collaborate on models, datasets, and applications. With over 1 million models and a rapidly growing collection of datasets and applications, we empower individuals and organizations to accelerate their machine learning projects.

---

## What We Offer
### **Community & Collaboration**
- **Explore AI Apps:** Collaborate on cutting-edge applications across various fields including text, image, video, audio, and 3D.
- **Public Models & Datasets:** Host and share unlimited public models and datasets on our platform, fostering innovation and experimentation.

### **Enterprise Solutions**
- Hugging Face provides enterprise-grade solutions tailored for teams, ensuring security, scalability, and dedicated support with plans starting at $20/user/month.

### **Open Source Tools**
- We invest in building foundational tools for machine learning, including libraries like Transformers for state-of-the-art ML in PyTorch, TensorFlow, and JAX, and Diffusers for advanced diffusion models.

---

## Our Customers
Over **50,000 organizations** trust Hugging Face, including major players in tech such as:
- **Google**
- **Amazon**
- **Meta**
- **Microsoft**
These organizations leverage our models and datasets to build innovative AI applications.

---

## Company Culture
At Hugging Face, we foster a collaborative and inclusive environment where creativity thrives. Our culture encourages open communication and community engagement, with a strong commitment to sustainability and ethical AI. We believe in empowering our team members to explore, innovate, and lead in the AI space together.

---

## Careers at Hugging Face
We are always looking for passionate and talented individuals to join our mission of democratizing AI. Whether you’re a seasoned professional or just starting your career in machine learning, we welcome you to explore opportunities in engineering, research, sales, and community engagement.

### **Join Us**
Explore our current job openings and see how you can contribute to the future of AI with Hugging Face.

---

## Get Started Today
Whether you're looking to explore AI models, collaborate on datasets, or join our community, **Hugging Face** is the platform for you. 

- **Sign Up** to start your journey with us.
- Explore our [Documentation](#) and [Blog](#) for resources and insights.

**Hugging Face – Together, we are shaping the future of AI.**

---

For more information, visit us at [HuggingFace.com](https://huggingface.co) or connect with us on [Twitter](https://twitter.com/huggingface) and [LinkedIn](https://www.linkedin.com/company/huggingface/).

--- 

*This brochure is intended for prospective customers, investors, and recruits interested in understanding Hugging Face's offerings, culture, and career opportunities.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Overview
**Hugging Face** is at the forefront of the artificial intelligence (AI) community, dedicated to building the future of technology. With a platform that fosters collaboration, researchers, developers, and enthusiasts come together to create and share machine learning models, datasets, and applications. Hugging Face is not just a tool; it is a thriving ecosystem that empowers the machine learning community.

---

## Our Offerings
- **Models:** Access a vast library of over **1 million models** that cater to a diverse range of AI applications.
- **Datasets:** Browse through **250,000 datasets** to find the right data for your machine learning projects.
- **Spaces:** Discover and create apps using our **400,000 applications**, running various models and facilitating unique user experiences.
- **Enterprise Solutions:** Tailored services for organizations looking to accelerate AI adoption, offering enhanced security, priority support, and dedicated resources.

---

## Customers
Hugging Face serves over **50,000 organizations**, including prominent companies such as:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**
- **Intel**

These organizations leverage Hugging Face’s open-source tools and enterprise solutions to drive innovation and efficiency in machine learning.

---

## Company Culture
At Hugging Face, we pride ourselves on being part of a **collaborative community**. Here, everyone is encouraged to share their work, contribute to the open-source movement, and build their unique machine learning portfolios. Our culture emphasizes continuous learning, inclusivity, and the pursuit of cutting-edge advancements in AI.

- **Community-Centric:** We believe in the power of collaboration and support our users by providing a platform for sharing knowledge and tools.
- **Open Source Focus:** We build our foundational tools like **Transformers** and **Diffusers** with contributions from the community, allowing everyone to benefit from and participate in advancements in ML.

---

## Careers at Hugging Face
Join us in our mission to advance AI technology! We are constantly on the lookout for passionate individuals to join our team. If you’re interested in building the future of AI, check our **Jobs page** for current openings.

- **Dynamic Environment:** Collaborate with some of the brightest minds in AI.
- **Diversity:** We welcome talents from all backgrounds and encourage people to bring their unique perspectives to the table.

---

## Get Started
Whether you're developing innovative AI applications, seeking robust data solutions, or looking for an engaging career opportunity, **Hugging Face** is your go-to platform. Sign up today to take your first step into a thriving AI community.

**Visit us at:** [Hugging Face](https://huggingface.co)

Connect with us on social media:  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface/)  
- [Discord](https://discord.gg/huggingface)  

---

### Join Us in Building the Future of AI!

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Pioneering the Future of AI

Welcome to **Hugging Face**, the leading AI community dedicated to building the future through collaboration, innovation, and open-source principles. Our platform serves as a vibrant ecosystem where machine learning enthusiasts, researchers, and businesses can explore, create, and co-develop groundbreaking AI models, datasets, and applications.

## Company Overview

- **Founded:** Hugging Face has emerged as a key player in the AI sector, growing rapidly to meet the evolving demands of machine learning and artificial intelligence.
- **Mission:** To democratize AI through accessible open-source tools and foster a thriving community centered on collaboration.

## Our Platform

At Hugging Face, we provide a state-of-the-art collaborative platform where over **50,000 organizations** leverage our tools for machine learning. Some highlights include:

- **1M+ Models:** Access a vast library of models with diverse applications.
- **250K+ Datasets:** Explore an extensive selection of datasets tailored for various ML tasks.
- **Spaces:** Create and run applications seamlessly using our optimized infrastructure.
- **Enterprise Solutions:** Tailored options offering advanced security, priority support, and dedicated resources for organizations.

### Customer Success Stories

Our platform is trusted by numerous world-leading organizations, including:
- **Microsoft**
- **Google**
- **Amazon**
- **AI at Meta**
- **Intel**

These collaborations highlight the effectiveness and reliability of Hugging Face in various applications of AI and machine learning.

## Company Culture

Hugging Face thrives on an inclusive, innovative, and community-driven culture. We believe in:
- **Collaboration:** Encouraging users to build together, combining collective intelligence to solve complex problems.
- **Openness:** Staying true to our open-source roots while providing comprehensive documentation and support for all users.
- **Empowerment:** Supporting team members and users to build their AI portfolios and actively contribute to the community.

### Team Environment

Join a team of passionate members working together to push the boundaries of AI. We value diversity, creativity, and a strong sense of community. Our collaborative work environment promotes knowledge sharing and growth.

## Careers at Hugging Face

At Hugging Face, we are continually looking for talented individuals to join our mission. If you are passionate about AI and want to make a significant impact in a dynamic team, consider a career with us. We offer various positions across technical, operational, and creative roles.

- **Current Openings:** Visit our [Jobs Page](https://huggingface.co/jobs) to explore exciting career opportunities.

## Join Us!

Whether you are a developer, researcher, or business looking to leverage AI, Hugging Face is your platform. Sign up today to gain access to unique models, high-quality datasets, and a supportive community that is shaping the future of machine learning.

### Contact Us

For inquiries or partnerships, please reach out via our [website](https://huggingface.co). Together, let’s build the future of AI! 

---

*Join the Hugging Face community today and be a part of the AI revolution.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>